Dueling, Double Q Network to solve simple game. Much of the code was taken from https://medium.com/@awjuliani/simple-reinforcement-learning-with-tensorflow-part-4-deep-q-networks-and-beyond-8438a3e2b8df

Also added coordConv to the input images per https://arxiv.org/abs/1807.03247

In [1]:
!apt-get install -y python-opengl
!pip install gym[atari]
!pip install JSAnimation

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  freeglut3 libglu1-mesa
Suggested packages:
  python-tk python-numpy libgle3
The following NEW packages will be installed:
  freeglut3 libglu1-mesa python-opengl
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 791 kB of archives.
After this operation, 6260 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian stretch/main amd64 freeglut3 amd64 2.8.1-3 [123 kB]
Get:2 http://deb.debian.org/debian stretch/main amd64 libglu1-mesa amd64 9.0.0-2.1 [168 kB]
Get:3 http://deb.debian.org/debian stretch/main amd64 python-opengl all 3.1.0+dfsg-1 [501 kB]
Fetched 791 kB in 0s (7118 kB/s)       
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package freeglut3:amd64.
(Reading database ... 35423 files and directories currently installed.)
Prepari

In [2]:
# Taken from https://arxiv.org/abs/1807.03247
def addCoords(image):
    n_rows, n_columns, _ = image.shape
    rows = np.matmul(np.reshape(np.array(range(n_rows)), [n_rows, 1]), np.ones([1, n_columns]))
    columns = np.matmul(np.ones([n_rows, 1]), np.reshape(np.array(range(n_columns)), [1, n_columns]))
    rows = (rows - (n_rows - 1) / 2) / ((n_rows -1 ) / 2)
    rows = np.expand_dims(rows, 2)
    columns = (columns - (n_columns - 1) / 2) / ((n_columns - 1) / 2)
    columns = np.expand_dims(columns, 2)

    return np.concatenate([image, rows, columns], axis=2)


In [3]:
import numpy as np
import random
import itertools
import scipy.misc
import matplotlib.pyplot as plt


class gameOb():
    def __init__(self,coordinates,size,intensity,channel,reward,name):
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.size = size
        self.intensity = intensity
        self.channel = channel
        self.reward = reward
        self.name = name
        
class GameEnv():
    class ActionSpace(object):
        
        def __init__(self, n):
            self.n = n
    def __init__(self,partial,size):
        self.sizeX = size
        self.sizeY = size
        self.actions = 4
        self.action_space = object()
        self.action_space = self.ActionSpace(self.actions)
        self.objects = []
        self.partial = partial
        a = self.reset()

        
        
    def reset(self):
        self.objects = []
        hero = gameOb(self.newPosition(),1,1,2,None,'hero')
        self.objects.append(hero)
        bug = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug)
        hole = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole)
        bug2 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug2)
        hole2 = gameOb(self.newPosition(),1,1,0,-1,'fire')
        self.objects.append(hole2)
        bug3 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug3)
        bug4 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug4)
        state = self.renderEnv()
        self.state = state
        return state

    def moveChar(self,direction):
        # 0 - up, 1 - down, 2 - left, 3 - right
        hero = self.objects[0]
        heroX = hero.x
        heroY = hero.y
        penalize = 0
        if direction == 0 and hero.y >= 1:
            hero.y -= 1
        if direction == 1 and hero.y <= self.sizeY-2:
            hero.y += 1
        if direction == 2 and hero.x >= 1:
            hero.x -= 1
        if direction == 3 and hero.x <= self.sizeX-2:
            hero.x += 1     
        if hero.x == heroX and hero.y == heroY:
            penalize = 0
        self.objects[0] = hero
        return penalize
    
    def newPosition(self):
        iterables = [ range(self.sizeX), range(self.sizeY)]
        points = []
        for t in itertools.product(*iterables):
            points.append(t)
        currentPositions = []
        for objectA in self.objects:
            if (objectA.x,objectA.y) not in currentPositions:
                currentPositions.append((objectA.x,objectA.y))
        for pos in currentPositions:
            points.remove(pos)
        location = np.random.choice(range(len(points)),replace=False)
        return points[location]

    def checkGoal(self):
        others = []
        for obj in self.objects:
            if obj.name == 'hero':
                hero = obj
            else:
                others.append(obj)
        ended = False
        for other in others:
            if hero.x == other.x and hero.y == other.y:
                self.objects.remove(other)
                if other.reward == 1:
                    self.objects.append(gameOb(self.newPosition(),1,1,1,1,'goal'))
                else: 
                    self.objects.append(gameOb(self.newPosition(),1,1,0,-1,'fire'))
                return other.reward,False
        if ended == False:
            return 0.0,False

    def renderEnv(self):
        #a = np.zeros([self.sizeY,self.sizeX,3])
        a = np.ones([self.sizeY+2,self.sizeX+2,3])
        a[1:-1,1:-1,:] = 0
        hero = None
        for item in self.objects:
            a[item.y+1:item.y+item.size+1,item.x+1:item.x+item.size+1,item.channel] = item.intensity
            if item.name == 'hero':
                hero = item
        if self.partial == True:
            a = a[hero.y:hero.y+3,hero.x:hero.x+3,:]
        b = scipy.misc.imresize(a[:,:,0],[84,84,1],interp='nearest')
        c = scipy.misc.imresize(a[:,:,1],[84,84,1],interp='nearest')
        d = scipy.misc.imresize(a[:,:,2],[84,84,1],interp='nearest')
        a = np.stack([b,c,d],axis=2)
        return addCoords(a)

    def step(self,action):
        penalty = self.moveChar(action)
        reward,done = self.checkGoal()
        state = self.renderEnv()
        if reward == None:
            print(done)
            print(reward)
            print(penalty)
            return state,(reward+penalty),done, None
        else:
            return state,(reward+penalty),done, None

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [ ]:
class QNetwork(object):
    
    def __init__(self, a_size, h_size, scope, input_dims):
        
        self.a_size = a_size
        self.h_size = h_size
        self.scope = scope
        self.input_dims = list(input_dims)
        self.trainer = tf.train.AdamOptimizer(learning_rate=.0001)

        with tf.variable_scope(self.scope):
            self.input = tf.placeholder(
                shape=[None] + self.input_dims,
                dtype=tf.float32
            )
            
            self.conv1 = slim.conv2d(
                self.input,
                kernel_size=[3, 3],
                num_outputs=32,
                stride=[1, 1],
                padding="VALID"
            )

            self.conv2 = slim.conv2d(
                self.conv1,
                kernel_size=[4, 4],
                num_outputs=64,
                stride=[2, 2],
                padding="VALID"
            )
            
            self.conv3 = slim.conv2d(
                self.conv2,
                kernel_size=[8, 8],
                num_outputs=64,
                stride=[4, 4],
                padding="VALID"
            )
            
            
            self.conv4 = slim.conv2d(
                self.conv3,
                kernel_size=[7, 7],
                num_outputs=h_size,
                stride=[1, 1],
                padding="VALID"
            )
            
            self.A_stream, self.V_stream = tf.split(
                self.conv4,
                2,
                axis=3
            )

            """
            self.advantage = slim.fully_connected(
                tf.layers.flatten(self.A_stream),
                self.a_size
            )
            
            self.value = slim.fully_connected(
                tf.layers.flatten(self.V_stream),
                1
            )
            """
            self.streamA = tf.layers.flatten(self.A_stream)
            self.streamV = tf.layers.flatten(self.V_stream)
            xavier_init = tf.contrib.layers.xavier_initializer()
            self.AW = tf.Variable(xavier_init([self.h_size * 9 // 2, self.a_size]))
            self.VW = tf.Variable(xavier_init([self.h_size * 9 // 2, 1]))
            self.advantage = tf.matmul(self.streamA, self.AW)
            self.value = tf.matmul(self.streamV, self.VW)

            self.Q = (self.advantage - tf.reduce_mean(self.advantage, 1, keep_dims=True)) + self.value
            self.prediction = tf.argmax(self.Q, 1)
            
            self.target_Q = tf.placeholder(
                shape=[None],
                dtype=tf.float32
            )
            
            self.action = tf.placeholder(
                shape=[None],
                dtype=tf.int32
            )
            
            action_onehot = tf.one_hot(self.action, self.a_size)
            self.loss = tf.reduce_mean(tf.square(
                tf.reduce_sum(
                    action_onehot * self.Q, 1
                ) - self.target_Q
            ))

            self.minimize = self.trainer.minimize(self.loss)
            
            

In [ ]:
def update_ops(from_scope, to_scope, tau):
    
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)
    
    ops = []
    for from_var, to_var in zip(from_vars, to_vars):
        op = to_var.assign(to_var.value() * (1 - tau) + from_var.value() * tau)
        ops.append(op)

    return ops

class ExperienceBuffer(object):
    
    def __init__(self, buffer_size=50000):
        
        self.buffer_size = buffer_size
        self._buffer = []
        
    def reset(self):
        
        self._buffer = []
 
    def sample(self, size):
        
        return random.sample(self._buffer, size)

    def add(self, experiences):
        
        e_length = len(experiences)
        current_size = len(self._buffer)
        if e_length + current_size > self.buffer_size:
            self._buffer = self._buffer[e_length + current_size - self.buffer_size: ] \
                + experiences
        else:
            self._buffer.extend(experiences)

In [ ]:
class Worker(object):
    
    MAX_EPISODE_LENGTH = 50
    PRE_TRAIN_STEPS = 10000
    BASE_E = .1
    EXTRA_E = 1 - BASE_E
    TRAIN_STEPS = 15000
    BATCH_SIZE = 32
    GAMMA = .99
    TAU = .001

    def __init__(self, sess, number_episodes=10000):
        
        self.env = GameEnv(partial=False, size=5)
        self.sess = sess
        self.buffer = ExperienceBuffer()
        self.number_episodes = number_episodes
        input_dims = self.env.reset().shape

        self.main_network = QNetwork(
            a_size=self.env.action_space.n,
            h_size=512,
            scope="main",
            input_dims=input_dims
        )
        
        self.target_network = QNetwork(
            a_size=self.env.action_space.n,
            h_size=512,
            scope="target",
            input_dims=input_dims
        )
        
        self.update_target_vars = update_ops("main", "target", self.TAU)

    def run(self):

        global_step = 0
        episode_rewards = []
        e = None
        for i in range(self.number_episodes):
            if global_step % 250 == 0:
                print ("Episode: {} Reward: {}".format(i, np.mean(episode_rewards[-25: ])))
            state = self.env.reset()
            episode_reward = 0
            for step in range(self.MAX_EPISODE_LENGTH):
                e = 1 - self.EXTRA_E * np.max(
                    [np.min([self.TRAIN_STEPS + self.PRE_TRAIN_STEPS, global_step]) - self.PRE_TRAIN_STEPS, 0]
                ) / self.TRAIN_STEPS
                global_step += 1
                Q, prediction = self.sess.run(
                    [
                        self.main_network.Q,
                        self.main_network.prediction
                    ],
                    feed_dict={
                        self.main_network.input: [state]
                    }
                )

                if np.random.rand(1) < e:
                    action = np.random.randint(self.env.action_space.n)
                else:
                    action = prediction
                
                state1, reward, done, _ = self.env.step(action)
                self.buffer.add([(state, action, state1, reward, done)])

                episode_reward += reward
                state = state1
                if global_step % 4 == 0 and global_step > self.PRE_TRAIN_STEPS:
                    experiences = self.buffer.sample(self.BATCH_SIZE)
                    states = [exp[0] for exp in experiences]
                    actions = [exp[1] for exp in experiences]
                    next_states = [exp[2] for exp in experiences]
                    rewards = np.array([exp[3] for exp in experiences])
                    dones = np.array([exp[4] for exp in experiences])

                    next_actions = self.sess.run(
                        self.main_network.prediction,
                        feed_dict={
                            self.main_network.input: next_states
                        }
                    )
                    
                    next_qs = self.sess.run(
                        self.target_network.Q,
                        feed_dict={
                            self.target_network.input: next_states
                        }
                    )
                    
                    target_qs = rewards + self.GAMMA * (1.0 - dones) * next_qs[range(self.BATCH_SIZE), next_actions]
                    
                    self.sess.run(
                        [self.main_network.minimize],
                        feed_dict={
                            self.main_network.input: states,
                            self.main_network.target_Q: target_qs,
                            self.main_network.action: actions
                        }
                    )
                    
                    self.sess.run(
                        self.update_target_vars
                    )

                if done:
                    break
            episode_rewards.append(episode_reward)

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    worker = Worker(sess=sess)
    sess.run(tf.global_variables_initializer())
    worker.run()

In [ ]:
tf.layers